In [75]:
from lexnorm.definitions import DATA_PATH
import os
import pandas as pd

train = pd.read_csv(os.path.join(DATA_PATH, "hpc/train_annotated.txt"), index_col=0)
dev = pd.read_csv(os.path.join(DATA_PATH, "hpc/dev_annotated_with_train_info.txt"), index_col=0)

In [76]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [78]:
train.fillna(0, inplace=True)
dev.fillna(0, inplace=True)
X_train = train.drop(["correct", "raw_tok_index", "gold"], axis=1)
X_dev = dev.drop(["correct", "process", "tweet", "tok", "gold"], axis=1)
y_train = train["correct"]
y_dev = dev["correct"]

In [79]:
from sklearn import preprocessing

feature_vals = X_train.values
scaler = preprocessing.StandardScaler().fit(feature_vals)
feature_vals = scaler.transform(feature_vals)
X_train[:] = feature_vals

feature_vals = X_dev.values
scaler = preprocessing.StandardScaler().fit(feature_vals)
feature_vals = scaler.transform(feature_vals)
X_dev[:] = feature_vals

In [5]:
svm = LinearSVC(class_weight="balanced", verbose=1).fit(X_train, y_train.values.ravel())

[LibLinear]....................................................................................................
optimization finished, #iter = 1000

Using -s 2 may be faster (also see FAQ)

Objective value = -77126.360842
nSV = 259470


/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [11]:
from joblib import dump, load
dump(svm, os.path.join(DATA_PATH, "../models/svm.joblib"))
test = load(os.path.join(DATA_PATH, "../models/svm.joblib"))
test.coef_

array([[ 0.17172243, -0.13962874, -0.20333317,  0.12403685,  0.00285971,
         1.8674295 , -0.0935477 ,  0.16077809, -0.05016316,  0.14621954,
        -0.64900722, -0.14582517,  0.        ,  0.08600042]])

In [80]:
preds = svm.decision_function(X_dev)

In [81]:
svm.score(X_dev, y_dev)

0.9983700596728575

In [127]:
X_train

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length
nah,2.656120,3.877197,-2.593884,-0.094405,-0.046030,-0.048241,-0.268871,-4.348835,-1.763720,-3.155060,-1.057043,0.345895,0.0,1.361154
oh,2.745444,2.535195,-2.593884,-0.094405,-0.046030,-0.048241,-0.268871,0.229947,-2.095137,-3.155060,-1.057043,0.345895,0.0,1.361154
uh,2.624856,5.219199,-2.593884,-0.094405,-0.046030,-0.048241,-0.268871,0.229947,-2.095137,-3.155060,-1.057043,0.345895,0.0,1.361154
ye ah,-1.138580,-0.148809,-2.593884,-0.094405,21.724887,-0.048241,1.360023,-4.348835,-1.100884,0.316951,-1.057043,0.345895,0.0,1.361154
ye-ah,-1.138580,-0.148809,-2.593884,-0.094405,-0.046030,-0.048241,1.767247,-4.348835,-1.100884,0.316951,-1.057043,0.345895,0.0,1.361154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
upward,0.569510,-0.148809,0.385522,-0.094405,-0.046030,-0.048241,-0.268871,0.229947,-0.769467,0.316951,-0.688526,0.345895,0.0,-0.462068
upwardly,0.702616,-0.148809,0.385522,-0.094405,-0.046030,-0.048241,-0.268871,0.229947,-0.106631,0.316951,-0.688526,0.345895,0.0,-0.462068
upwards,0.833669,-0.148809,0.385522,-0.094405,-0.046030,-0.048241,-0.268871,0.229947,-0.438049,0.316951,-0.688526,0.345895,0.0,-0.462068
us,1.122110,-0.148809,-2.593884,-0.094405,-0.046030,-0.048241,2.988917,0.229947,-2.095137,-3.155060,-0.688526,0.345895,0.0,-0.462068


In [83]:
dev["preds"] = preds
dev

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length,correct,process,tweet,tok,gold,preds
an,0.377180,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2,0.0,72.0,1.0,1.0,3.0,0.0,35,0,2,can,-1.434081
ca,0.255824,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2,0.0,72.0,1.0,1.0,3.0,0.0,35,0,2,can,-1.568795
cab,0.261057,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3,0.0,72.0,1.0,1.0,3.0,0.0,35,0,2,can,-1.469996
cam,0.299136,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3,0.0,72.0,1.0,1.0,3.0,0.0,35,0,2,can,-1.476411
can,1.000000,0.0,1.0,1.0,0.0,72.0,0.0,1.0,3,1.0,72.0,1.0,1.0,3.0,1.0,35,0,2,can,9.580333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lulu,0.401415,0.0,0.0,0.0,0.0,0.0,14.0,1.0,4,0.0,205.0,1.0,1.0,3.0,0.0,32,9,195,lol,-2.435480
pol,0.230619,0.0,0.0,0.0,0.0,0.0,9.0,1.0,3,0.0,205.0,1.0,1.0,3.0,0.0,32,9,195,lol,-2.371043
smh,0.896277,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,205.0,1.0,1.0,3.0,0.0,32,9,195,lol,-2.399906
sol,0.285978,0.0,0.0,0.0,0.0,0.0,3.0,1.0,3,0.0,205.0,1.0,1.0,3.0,0.0,32,9,195,lol,-2.104146


In [112]:
# get top prediction for each raw token
predictions = dev.sort_values("preds", ascending=False).drop_duplicates(["process", "tweet", "tok"])
pred_tokens = predictions.sort_values(["process", "tweet", "tok"])["gold"].tolist()
pred_tokens

['brother',
 'get',
 'out',
 'your',
 'feelings',
 'lol',
 'manan',
 'dund',
 'xaragdax',
 'ter',
 'uuliin',
 'oroid',
 'minii',
 'aav',
 'xezee',
 'neg',
 'cagt',
 'zogsoj',
 'baisan',
 'photo',
 'by',
 'why',
 'these',
 'niggers',
 'think',
 'they',
 'doing',
 'something',
 "it's",
 'about',
 'more',
 'than',
 'number',
 'brother',
 'and',
 "i'm",
 'not',
 'talking',
 'about',
 'statistics',
 "i'm",
 'talking',
 'about',
 'skill',
 'level',
 'yes',
 'omg',
 'i',
 "don't",
 'want',
 'him',
 'feeling',
 'unappreciated',
 'or',
 'stuff',
 'like',
 'that',
 'lmao',
 'the',
 'whole',
 'time',
 'actually',
 'we',
 'need',
 'to',
 'start',
 'our',
 'own',
 'team',
 "y'all",
 'were',
 'wilding',
 'last',
 'night',
 'yoh',
 'niya',
 'ja',
 'saying',
 'kau',
 'aku',
 'with',
 'couple',
 'is',
 'actually',
 'sweet',
 'k',
 'michelle',
 'met',
 'you',
 'match',
 'that',
 'fonk',
 'rana',
 'samaha',
 'ignore',
 'all',
 'the',
 'criticism',
 'a',
 'spotlight',
 'can',
 'never',
 'see',
 'the',
 's

In [92]:
# pretty good job!
predictions.groupby("correct").size()

correct
0.0     407
1.0    6469
dtype: int64

In [93]:
from lexnorm.data.normEval import loadNormData

raw, norm = loadNormData(os.path.join(DATA_PATH, "raw/dev.norm"))

In [125]:
from lexnorm.models.filtering import is_eligible

pred_tokens_iter = iter(pred_tokens)

pred_tweets = []

for tweet in raw:
    pred_tweet = []
    for tok in tweet:
        if is_eligible(tok):
            pred_tweet.append(next(pred_tokens_iter))
        else:
            pred_tweet.append(tok)
    pred_tweets.append(pred_tweet)

In [126]:
from lexnorm.data.normEval import evaluate

evaluate(raw, norm, pred_tweets)

Baseline acc.(LAI): 93.10
Accuracy:           99.97
ERR:                99.53


(0.9309630275929763, 0.9996728105573127, 0.9952606635071095)